In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from string import punctuation
import numpy as np
import pandas as pd
import pickle, re
from typing import List, Dict, Any
import datetime

In [2]:
STORES_DICT = {
    "the_bar": "The Bar",
    "casadabebida": "Casa da Bebida"
}

In [3]:
def get_meta_info(product: str, store: str) -> Dict:
  sku = None
  brand = None
  description = None
  availability = None
  price_currency = None
  price_amount = None
  url = None

  if all_products[product][store] is not None:
    url = all_products[product][store]
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    if store=="the_bar":
      sku = soup.find("meta", property="product:sku")
      brand = soup.find("meta", property="product:brand")
      price_currency = soup.find("meta", property="product:price:currency")
      availability = soup.find("meta", property="product:availability")
      price_amount = soup.find("meta", property="product:price:amount")

      sku = sku['content'] if sku else None
      brand = brand['content'] if brand else None
      price_currency = price_currency['content'] if price_currency else None
      availability = availability['content'] if availability else None
      price_amount = float(price_amount['content']) if price_amount else None

    elif store=="casadabebida":
      sku = soup.find(itemprop="sku").get_text()
      brand = soup.find(itemprop="brand").get_text().strip()

      description = soup.find("meta", property="og:title")
      description = description['content'] if description else None

      price_currency = soup.find(itemprop="priceCurrency").get("content")
      availability = soup.find(itemprop="availability").get("content").split("/")[-1]
      price_amount = float(soup.find(itemprop="price").get_text())

  infos = {
      "sku": sku, 
      "marca": brand,
      "descricao": description,
      "disponibilidade": availability,
      "preco_moeda": price_currency,
      "preco_valor": price_amount,
      "loja": STORES_DICT[store],
      "data": datetime.datetime.now().isoformat(),
      "url": url
    }
  return infos

In [4]:
all_products = {
    "singleton_12y": {
        "the_bar": "https://br.thebar.com/whisky-singleton-of-dufftown-12-anos--750ml-gre30284/p",
        "casadabebida": "https://www.casadabebida.com.br/whisky/singleton-12-anos-of-glen-ord-700-ml/",
    },
    "glenkinchie_12y": {
        "the_bar": "https://br.thebar.com/whisky-glenkichie-12-anos--750ml-gre30252/p",
        "casadabebida": "https://www.casadabebida.com.br/whisky/glenkinchie-12-anos-750-ml/"
    },
    "glenfiddich_12y": {
        "the_bar": None,
        "casadabebida": "https://www.casadabebida.com.br/whisky/single-malt/whisky-glenfiddich-12-anos/"
    },
    "talisker_10y":  {
        "the_bar": "https://br.thebar.com/whisky-talisker-10-anos-750ml-gre30290/p",
        "casadabebida": "https://www.casadabebida.com.br/whisky/talisker-10-anos-750-ml/"
    },
    "johnnie_green": {
        "the_bar": "https://br.thebar.com/whisky-johnnie-walker-green-label--750ml-gre30263/p",
        "casadabebida": "https://www.casadabebida.com.br/whisky/whisky-johnnie-walker-green-label/"
    },
    "glenlivet_15y": {
        "the_bar": None,
        "casadabebida": "https://www.casadabebida.com.br/whisky/glenlivet-15-anos-750-ml/"
    },
    "laphroaig_10y": {
        "the_bar": None,
        "casadabebida": "https://www.casadabebida.com.br/whisky/whisky-laphroaig-10-anos/"
    },
    "macallan_sherryoak_12y": {
        "the_bar": None,
        "casadabebida": "https://www.casadabebida.com.br/whisky/macallan-sherry-oak-12-anos-700-ml/"
    },
    "macallan_18y": {
        "the_bar": None,
        "casadabebida": "https://www.casadabebida.com.br/whisky/whisky-the-macallan-18-anos/"
    }
  }

In [5]:
get_meta_info("glenkinchie_12y", "casadabebida")

{'sku': '873',
 'marca': 'Glenkinchie',
 'descricao': 'Whisky Glenkinchie 12 Anos 750 ml - Single Malt',
 'disponibilidade': 'InStock',
 'preco_moeda': 'BRL',
 'preco_valor': 351.41,
 'loja': 'Casa da Bebida',
 'data': '2022-04-01T14:37:01.505808',
 'url': 'https://www.casadabebida.com.br/whisky/glenkinchie-12-anos-750-ml/'}

In [6]:
data = [get_meta_info(x,'casadabebida') for x in all_products.keys()]

In [7]:
dataframe = pd.DataFrame(data=data, columns = ["marca", "descricao", "disponibilidade", "loja", "sku","preco_moeda", "preco_valor", "data", "url"])
dataframe

,marca,descricao,disponibilidade,loja,sku,preco_moeda,preco_valor,data,url
0,Singleton,Whisky Singleton 12 Anos Dufftown 700 ml,InStock,Casa da Bebida,874,BRL,189.91,2022-04-01T14:37:02.796013,https://www.casadabebida.com.br/whisky/singlet...
1,Glenkinchie,Whisky Glenkinchie 12 Anos 750 ml - Single Malt,InStock,Casa da Bebida,873,BRL,351.41,2022-04-01T14:37:03.498085,https://www.casadabebida.com.br/whisky/glenkin...
2,Glenfiddich,Whisky Glenfiddich 12 anos 750ml,InStock,Casa da Bebida,114,BRL,284.91,2022-04-01T14:37:04.273149,https://www.casadabebida.com.br/whisky/single-...
3,Talisker,Whisky Talisker 10 anos 750 ml,InStock,Casa da Bebida,872,BRL,379.91,2022-04-01T14:37:05.011156,https://www.casadabebida.com.br/whisky/taliske...
4,Johnnie Walker,Whisky Johnnie Walker Green Label 750 ml,InStock,Casa da Bebida,15,BRL,379.91,2022-04-01T14:37:05.785094,https://www.casadabebida.com.br/whisky/whisky-...
5,Glenlivet,Whisky Glenlivet 15 anos 750 ml,InStock,Casa da Bebida,1139,BRL,379.91,2022-04-01T14:37:06.609017,https://www.casadabebida.com.br/whisky/glenliv...
6,Laphroaig,Whisky Laphroaig 10 Anos 750 ml,InStock,Casa da Bebida,750,BRL,398.91,2022-04-01T14:37:07.347277,https://www.casadabebida.com.br/whisky/whisky-...
7,Macallan,Whisky Macallan Sherry Oak 12 anos 700 ml,InStock,Casa da Bebida,2744,BRL,906.21,2022-04-01T14:37:08.073078,https://www.casadabebida.com.br/whisky/macalla...
8,Macallan,Whisky The Macallan 18 anos - Double Cask 700 ml,InStock,Casa da Bebida,226,BRL,3061.76,2022-04-01T14:37:08.779426,https://www.casadabebida.com.br/whisky/whisky-...


In [8]:
dataframe.sort_values(by="preco_valor", ascending=True, inplace=True)

In [9]:
dataframe=dataframe.reset_index(drop=True)

In [10]:
dataframe

,marca,descricao,disponibilidade,loja,sku,preco_moeda,preco_valor,data,url
0,Singleton,Whisky Singleton 12 Anos Dufftown 700 ml,InStock,Casa da Bebida,874,BRL,189.91,2022-04-01T14:37:02.796013,https://www.casadabebida.com.br/whisky/singlet...
1,Glenfiddich,Whisky Glenfiddich 12 anos 750ml,InStock,Casa da Bebida,114,BRL,284.91,2022-04-01T14:37:04.273149,https://www.casadabebida.com.br/whisky/single-...
2,Glenkinchie,Whisky Glenkinchie 12 Anos 750 ml - Single Malt,InStock,Casa da Bebida,873,BRL,351.41,2022-04-01T14:37:03.498085,https://www.casadabebida.com.br/whisky/glenkin...
3,Talisker,Whisky Talisker 10 anos 750 ml,InStock,Casa da Bebida,872,BRL,379.91,2022-04-01T14:37:05.011156,https://www.casadabebida.com.br/whisky/taliske...
4,Johnnie Walker,Whisky Johnnie Walker Green Label 750 ml,InStock,Casa da Bebida,15,BRL,379.91,2022-04-01T14:37:05.785094,https://www.casadabebida.com.br/whisky/whisky-...
5,Glenlivet,Whisky Glenlivet 15 anos 750 ml,InStock,Casa da Bebida,1139,BRL,379.91,2022-04-01T14:37:06.609017,https://www.casadabebida.com.br/whisky/glenliv...
6,Laphroaig,Whisky Laphroaig 10 Anos 750 ml,InStock,Casa da Bebida,750,BRL,398.91,2022-04-01T14:37:07.347277,https://www.casadabebida.com.br/whisky/whisky-...
7,Macallan,Whisky Macallan Sherry Oak 12 anos 700 ml,InStock,Casa da Bebida,2744,BRL,906.21,2022-04-01T14:37:08.073078,https://www.casadabebida.com.br/whisky/macalla...
8,Macallan,Whisky The Macallan 18 anos - Double Cask 700 ml,InStock,Casa da Bebida,226,BRL,3061.76,2022-04-01T14:37:08.779426,https://www.casadabebida.com.br/whisky/whisky-...


In [11]:
dataframe = dataframe.set_index('data')

In [12]:
dataframe

,marca,descricao,disponibilidade,loja,sku,preco_moeda,preco_valor,url
data,,,,,,,,
2022-04-01T14:37:02.796013,Singleton,Whisky Singleton 12 Anos Dufftown 700 ml,InStock,Casa da Bebida,874,BRL,189.91,https://www.casadabebida.com.br/whisky/singlet...
2022-04-01T14:37:04.273149,Glenfiddich,Whisky Glenfiddich 12 anos 750ml,InStock,Casa da Bebida,114,BRL,284.91,https://www.casadabebida.com.br/whisky/single-...
2022-04-01T14:37:03.498085,Glenkinchie,Whisky Glenkinchie 12 Anos 750 ml - Single Malt,InStock,Casa da Bebida,873,BRL,351.41,https://www.casadabebida.com.br/whisky/glenkin...
2022-04-01T14:37:05.011156,Talisker,Whisky Talisker 10 anos 750 ml,InStock,Casa da Bebida,872,BRL,379.91,https://www.casadabebida.com.br/whisky/taliske...
2022-04-01T14:37:05.785094,Johnnie Walker,Whisky Johnnie Walker Green Label 750 ml,InStock,Casa da Bebida,15,BRL,379.91,https://www.casadabebida.com.br/whisky/whisky-...
2022-04-01T14:37:06.609017,Glenlivet,Whisky Glenlivet 15 anos 750 ml,InStock,Casa da Bebida,1139,BRL,379.91,https://www.casadabebida.com.br/whisky/glenliv...
2022-04-01T14:37:07.347277,Laphroaig,Whisky Laphroaig 10 Anos 750 ml,InStock,Casa da Bebida,750,BRL,398.91,https://www.casadabebida.com.br/whisky/whisky-...
2022-04-01T14:37:08.073078,Macallan,Whisky Macallan Sherry Oak 12 anos 700 ml,InStock,Casa da Bebida,2744,BRL,906.21,https://www.casadabebida.com.br/whisky/macalla...
2022-04-01T14:37:08.779426,Macallan,Whisky The Macallan 18 anos - Double Cask 700 ml,InStock,Casa da Bebida,226,BRL,3061.76,https://www.casadabebida.com.br/whisky/whisky-...


In [52]:
dataframe.index

Index(['2022-04-01T14:37:02.796013', '2022-04-01T14:37:04.273149',
       '2022-04-01T14:37:03.498085', '2022-04-01T14:37:05.011156',
       '2022-04-01T14:37:05.785094', '2022-04-01T14:37:06.609017',
       '2022-04-01T14:37:07.347277', '2022-04-01T14:37:08.073078',
       '2022-04-01T14:37:08.779426'],
      dtype='object', name='data')

In [53]:
old_df = pd.read_csv("../output/dataframe_casadabebida.csv", index_col=0, parse_dates=True, infer_datetime_format=True)

In [56]:
old_df

,marca,descricao,disponibilidade,loja,sku,preco_moeda,preco_valor,url
data,,,,,,,,
2022-04-01 11:56:13.450845,Singleton,Whisky Singleton 12 Anos Dufftown 700 ml,InStock,Casa da Bebida,874,BRL,189.91,https://www.casadabebida.com.br/whisky/singlet...
2022-04-01 11:56:14.943618,Glenfiddich,Whisky Glenfiddich 12 anos 750ml,InStock,Casa da Bebida,114,BRL,284.91,https://www.casadabebida.com.br/whisky/single-...
2022-04-01 11:56:14.191140,Glenkinchie,Whisky Glenkinchie 12 Anos 750 ml - Single Malt,InStock,Casa da Bebida,873,BRL,351.41,https://www.casadabebida.com.br/whisky/glenkin...
2022-04-01 11:56:15.728990,Talisker,Whisky Talisker 10 anos 750 ml,InStock,Casa da Bebida,872,BRL,379.91,https://www.casadabebida.com.br/whisky/taliske...
2022-04-01 11:56:16.482897,Johnnie Walker,Whisky Johnnie Walker Green Label 750 ml,InStock,Casa da Bebida,15,BRL,379.91,https://www.casadabebida.com.br/whisky/whisky-...
2022-04-01 11:56:17.251146,Glenlivet,Whisky Glenlivet 15 anos 750 ml,InStock,Casa da Bebida,1139,BRL,379.91,https://www.casadabebida.com.br/whisky/glenliv...
2022-04-01 11:56:17.946285,Laphroaig,Whisky Laphroaig 10 Anos 750 ml,InStock,Casa da Bebida,750,BRL,398.91,https://www.casadabebida.com.br/whisky/whisky-...
2022-04-01 11:56:18.780784,Macallan,Whisky Macallan Sherry Oak 12 anos 700 ml,InStock,Casa da Bebida,2744,BRL,906.21,https://www.casadabebida.com.br/whisky/macalla...
2022-04-01 11:56:19.503811,Macallan,Whisky The Macallan 18 anos - Double Cask 700 ml,InStock,Casa da Bebida,226,BRL,3061.76,https://www.casadabebida.com.br/whisky/whisky-...


In [57]:
import altair as alt
from vega_datasets import data
#alt.renderers.enable('altair_saver', fmts=['vega-lite', 'png'])

temps = data.seattle_temps()
temps.head()

,date,temp
0,2010-01-01 00:00:00,39.4
1,2010-01-01 01:00:00,39.2
2,2010-01-01 02:00:00,39.0
3,2010-01-01 03:00:00,38.9
4,2010-01-01 04:00:00,38.8


In [58]:
temps = temps[temps.date < '2010-01-15']


alt.Chart(temps).mark_line().encode(
    x='date:T',
    y='temp:Q'
)

alt.Chart(...)

In [59]:
old_df.index

DatetimeIndex(['2022-04-01 11:56:13.450845', '2022-04-01 11:56:14.943618',
               '2022-04-01 11:56:14.191140', '2022-04-01 11:56:15.728990',
               '2022-04-01 11:56:16.482897', '2022-04-01 11:56:17.251146',
               '2022-04-01 11:56:17.946285', '2022-04-01 11:56:18.780784',
               '2022-04-01 11:56:19.503811', '2022-04-02 12:04:35.016311',
               '2022-04-01 12:04:36.491235', '2022-04-01 12:04:35.748929',
               '2022-04-01 12:04:37.282103', '2022-04-01 12:04:38.023949',
               '2022-04-01 12:04:38.803978', '2022-04-01 12:04:39.519494',
               '2022-04-01 12:04:40.264905', '2022-04-01 12:04:41.009805',
               '2022-04-03 12:04:35.016311'],
              dtype='datetime64[ns]', name='data', freq=None)

In [60]:
plot_df = old_df.reset_index()
plot_df

,data,marca,descricao,disponibilidade,loja,sku,preco_moeda,preco_valor,url
0,2022-04-01 11:56:13.450845,Singleton,Whisky Singleton 12 Anos Dufftown 700 ml,InStock,Casa da Bebida,874,BRL,189.91,https://www.casadabebida.com.br/whisky/singlet...
1,2022-04-01 11:56:14.943618,Glenfiddich,Whisky Glenfiddich 12 anos 750ml,InStock,Casa da Bebida,114,BRL,284.91,https://www.casadabebida.com.br/whisky/single-...
2,2022-04-01 11:56:14.191140,Glenkinchie,Whisky Glenkinchie 12 Anos 750 ml - Single Malt,InStock,Casa da Bebida,873,BRL,351.41,https://www.casadabebida.com.br/whisky/glenkin...
3,2022-04-01 11:56:15.728990,Talisker,Whisky Talisker 10 anos 750 ml,InStock,Casa da Bebida,872,BRL,379.91,https://www.casadabebida.com.br/whisky/taliske...
4,2022-04-01 11:56:16.482897,Johnnie Walker,Whisky Johnnie Walker Green Label 750 ml,InStock,Casa da Bebida,15,BRL,379.91,https://www.casadabebida.com.br/whisky/whisky-...
5,2022-04-01 11:56:17.251146,Glenlivet,Whisky Glenlivet 15 anos 750 ml,InStock,Casa da Bebida,1139,BRL,379.91,https://www.casadabebida.com.br/whisky/glenliv...
6,2022-04-01 11:56:17.946285,Laphroaig,Whisky Laphroaig 10 Anos 750 ml,InStock,Casa da Bebida,750,BRL,398.91,https://www.casadabebida.com.br/whisky/whisky-...
7,2022-04-01 11:56:18.780784,Macallan,Whisky Macallan Sherry Oak 12 anos 700 ml,InStock,Casa da Bebida,2744,BRL,906.21,https://www.casadabebida.com.br/whisky/macalla...
8,2022-04-01 11:56:19.503811,Macallan,Whisky The Macallan 18 anos - Double Cask 700 ml,InStock,Casa da Bebida,226,BRL,3061.76,https://www.casadabebida.com.br/whisky/whisky-...
9,2022-04-02 12:04:35.016311,Singleton,Whisky Singleton 12 Anos Dufftown 700 ml,InStock,Casa da Bebida,874,BRL,159.91,https://www.casadabebida.com.br/whisky/singlet...


In [62]:
plot_df=plot_df[["data", "marca", "preco_valor"]]
plot_df

,data,marca,preco_valor
0,2022-04-01 11:56:13.450845,Singleton,189.91
1,2022-04-01 11:56:14.943618,Glenfiddich,284.91
2,2022-04-01 11:56:14.191140,Glenkinchie,351.41
3,2022-04-01 11:56:15.728990,Talisker,379.91
4,2022-04-01 11:56:16.482897,Johnnie Walker,379.91
5,2022-04-01 11:56:17.251146,Glenlivet,379.91
6,2022-04-01 11:56:17.946285,Laphroaig,398.91
7,2022-04-01 11:56:18.780784,Macallan,906.21
8,2022-04-01 11:56:19.503811,Macallan,3061.76
9,2022-04-02 12:04:35.016311,Singleton,159.91


In [63]:
chart = alt.Chart(plot_df).mark_line().encode(
    x='data:T',
    y='preco_valor:Q',
    color='marca'
).properties(
    title="Whisky Price Trends",
    width=840,
    height=320,
)
chart

alt.Chart(...)

In [ ]:
chart.save('chart_whisky.png')